## LSTM model example

In [68]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


In [69]:
type(text),len(text), text[1:30000]

(str,
 600893,
 'reface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the

In [70]:

# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


In [72]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [9]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Epoch 1/1
200278/200278 [==============================] - 167s 834us/step - loss: 1.8030
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the farse the sensitication of shills the sense the spiriting that it is the self-consequentice of the sense the condication of the preasent the sense the sensiticy and it is the consequent the decause the same in the harded the germant and the sense and something the decised the decisence of the have the sense and in the decisence of the dispending the germant attered, the sense the sense them 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for be are all the spirits, his much which he has the world will thing in musch and
conscience of something aw it is person every the formerity in histering in the germans in relation it is even spiriting is all the inselfaring much into a means of succosicititical sensitication 

KeyboardInterrupt: 

## EBS LSTM model (30 epoch)

In [3]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [10]:
from jamo import h2j, j2hcj, j2h

from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein
from soynlp.hangle import compose
from soynlp.hangle import decompose

import nltk
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.help import upenn_tagset
from nltk import Text

from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import numpy as np
import re
from collections import defaultdict
from string import punctuation
import sys
import os

In [5]:
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS
from PreProcessing.Opentext import get_EBS_entered
from PreProcessing.Opentext import get_STT1_Google_entered
from PreProcessing.Opentext import get_STT1_Transcribe_entered
from PreProcessing.find_common_part import find_common_part

0
0
0
1
2
5
4
8
['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [13]:
def decimalnumber_to_korean(m):
    answer = ''
    
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

# EBS 문장들 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)

    return ebs_alphabet_conversion

def insert_dot(text, dot):
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    



In [14]:
dic = []
for i in range(1, 27):
    n = '{0:05d}'.format(i)
    lec = 'lec' + n + '_EBS.txt'
    dic.append(lec)

texts = ''

for _ in dic:
    with open('data/'+ _ ,'r',encoding='utf-8') as r:
        texts += r.read()

In [15]:
EBS = texts
ko_pre_EBS = EBS_preprocessing(EBS)
ko_pre_EBS

'반갑습니다, 이비에스 친구 여러분들. 여러분들과 함께 중학뉴런수학삼하를 강의하게 된 손석민입니다.이학년 뉴런을 통해서도 만났었는데요. 여러분들, 이렇게 다시 만나게 되어서 정말 반갑습니다.앞으로 삼하 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요.오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 삼학년 과정이, 하 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다.일단 삼학년 과정 같은 경우에는 크게 삼개의 대단원으로 이루어져 있는데요. 첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질, 세 번째가 통계. 이렇게 이루어져 있습니다.우리가 일학년하도 마찬가지고 이학년하도 그래 왔듯, ‘주로 도형에 대한 이야기가 삼하에서 많이 다루어지는구나.’ 라는 걸 생각하면 될 테고요.오늘은 그 첫 시간이니까 ‘삼각비.’ ‘삼각’ 알아요. ‘비’ 알아요. 근데 이제 합쳐져 있는 ‘삼각비’라는 새로운 용어가 나왔죠.그럼 이 개념을 공부하기에 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다. 오늘 그 첫 시간, 함께 시작해 보도록 할 텐데요. 중학교 삼학년 수학 손샘과 끝냅시다오늘 공부할 내용, 바로 두 가지 개념인데요. 오늘 두 가지 개념 첫 번째 제목이 굉장히 길어요. ‘닮은 직각삼각형에서 대응변의 길이의 비’라는 것이 있습니다.그다음 두 번째 개념이 바로 뭐냐면, ‘삼각비’라는 건데 삼각비의 뜻이겠죠? 이 첫 번째 개념은, 앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자는, ‘삼각비를 공부하기에 앞선 징검다리 내용이다.’ 라고 여러분들이 받아들이시면 됩니다.그렇다면 ‘닮은 직각삼각형에서’라고 했으니까 닮은 직각삼각형을 그려 봐야 되겠죠? 닮은 직각삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면우리는 이렇게 그리면 될 거 같아요. 이렇게 한번 그려 보겠습니다, 이렇게. 여기에서부터 수선을 이렇게 그어 보겠습니

In [16]:
# linesplitted_EBS = ' '.join(insert_dot(ko_pre_EBS,'.')).split('. ')
# nwhites_EBS = [''.join(_.split()) for _ in linesplitted_EBS]

# morphed_EBS = list()
# for _ in nwhites_EBS:
#     morphs = Okt().pos(_)
#     morphed_EBS.append(' '.join([(_[0]) for _ in morphs if _[1] in ['Noun', 'Verb', 'Adjective']]))
# morphed_EBS


In [17]:
## 남아있는 알파벳 제거
notkorpat = re.compile(r'[a-zA-Z]+')
ko_pre_EBS = notkorpat.sub(' ', ko_pre_EBS)

ko_pre_EBS = str(ko_pre_EBS)
type(ko_pre_EBS), len(ko_pre_EBS)


(str, 367508)

In [18]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(ko_pre_EBS) - maxlen, step):
    sentences.append(ko_pre_EBS[i: i + maxlen])
    next_chars.append(ko_pre_EBS[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(ko_pre_EBS)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 122483
고유한 글자: 772
벡터화...


In [99]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [100]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [101]:
# import random
# import sys

# random.seed(42)
# start_index = random.randint(0, len(ko_pre_EBS) - maxlen - 1)

# # 30 에포크 동안 모델을 훈련합니다
# for epoch in range(1, 30):
#     print('epoch', epoch)
#     # 데이터에서 한 번만 반복해서 모델을 학습합니다
#     model.fit(x, y, batch_size=128, epochs=1)

#     # 무작위로 시드 텍스트를 선택합니다
#     seed_text = ko_pre_EBS[start_index: start_index + maxlen]
#     print('--- 시드 텍스트: "' + seed_text + '"')

#     # 여러가지 샘플링 온도를 시도합니다
#     for temperature in [0.2, 0.5, 1.0, 1.2]:
#         print('------ 온도:', temperature)
#         generated_text = seed_text
#         sys.stdout.write(generated_text)

#         # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
#         for i in range(400):
#             # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
#             sampled = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(generated_text):
#                 sampled[0, t, char_indices[char]] = 1.

#             # 다음 글자를 샘플링합니다
#             preds = model.predict(sampled, verbose=0)[0]
#             next_index = sample(preds, temperature)
#             next_char = chars[next_index]

#             generated_text += next_char
#             generated_text = generated_text[1:]

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

epoch 1
Epoch 1/1
122483/122483 [==============================] - 91s 740us/step - loss: 2.6180
--- 시드 텍스트: "렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨"
------ 온도: 0.2
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨분의씨에이치가 이분의루트삼이니까 이분의루트삼 나누 삼각형에이비씨에서 이 길이가 이분의일 곱하기 사인 삼십도도 이렇게 여러분들이 구할 수 있어요. 그래서 이 각이 삼십도 이렇게 여러분들이 구할 수 있어요. 그러면 이 길이가 이분의루트삼 기준각이 이분의루트삼 이렇게 여러분들이 구할 수 있다는 얘기야. 그렇죠? 그렇죠? 그렇죠? 그러면 이 길이가 이 길이가 이분의루트삼 나누 삼각형에이비씨에서 이 길이가 이분의일 곱하기 사인 삼십도도 이렇게 여러분들이 구할 수 있어요. 그렇죠? 그러면 이 길이가 이 길이가 이분의일 곱하기 두 변의 길이의 곱의 길이의 곱하기 사인 엑스 이렇게 여러분들이 구할 수 있다는 얘기야. 그러면 이 각이 삼각형의 넓이가 이렇게 여러분들이 구할 수 있어요. 그래서 이 각이 삼십도 이렇게 여러분들
------ 온도: 0.5
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨의 길이가 구하고 싶어요. 그래서 이 길이가 이렇게 여러분들이 구할 수 있으니까 이 길이를 구하라는 얘기예요. 그다음 문제 이제 여러분들이 좀 풀이 문제도 여러분들이 답을 구할 수 있겠죠? 이 삼각형 에이에이치는 여기 있어요. 그래서 이 삼각형에이치는 삼각형 에이에이치를 구할 거야. 그렇죠? 그렇죠? 그러면 이렇게 여러분들이 구할 수 있으니까 이 길이가 루트삼분의삼이니까, 이렇게 여러분들이 코사인 사십오도는 이분의루트삼이라고 이렇게 여러분들이 구할 수 있여 수 있는데 중을 이 변이 니까 사분

렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨는 높이예요, 일에이디죠.래러기 때문에 마주 보이는 게 높이가 하고 밑변분의높이가 사인 엑스. 빗변분의높이. 이렇게 생각각보다각 사인 삼십도, 사십오도. 영도가 돼요. 누구다? 오에이분의삼이니까. 그러면 이렇게 될 수 있는 거예요. 쓸 수 있죠?사번 문제는 이거 이에서 그거래, 이렇게 했었던 한 변이 에이디죠. 코사인의 값이고, 삼각비. 누구를 구하래? 에이비예요. 이렇게 가 볼게요. 그러면 이 각 것도 이 높이를 이 용을 알고  더 ’가 되냐 하는 거 그 중에 이걸 밑변, 나머지 두 변 빠들이게 제일 건, 마지막체 가 되는 거리가 주어져 있어. 그렇죠? 예각이해야 되겠습니다. 두 번째 각비분은 팔한 있어. 그런데 삼각형 에이비씨에서 공부할 내용에이의 예각을 사인, 코사인을 이렇게 주어져 있어야 되겠네요.탄
------ 온도: 1.2
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨를 잘 구 아로 의 제곱이가 여러분들이 는 오이다. 됐고요.백팔십도 둘은 들기 양로 기준각을 의제곱해 분억선.모든또 일단 이 걸 ? 삼십입니다.그런데 ‘이 길이 지었죠? 뭐라는 걸 될 테고,에이에서. 선생님이. 그렇죠? 씨에이치진 정리를 한다.의 했잖아요. 그런데 그러면 올 게 있어요요. 기준각으로 대라것 오렇죠. 에이비를 구해 보도록 아습뭐 구하네. 기준 각에서 배우는 건 오늘 보세요. 개는 수분의 잘 삼분의육이다.이렇게 난른잡두 왜 풀어야?비? 사 이해야는. 하나에가 우리 얘부치까지왜 형예요? 어떻게 들어해야 되겠구할 수 있으면 된다는 걸 끝니까 성되는 코사인 높이더하기삼배곱하기육는사인, 이제 오분의첫 인 직각삼각형을 나반서 이거 합져 어떻게 선생님해 볼까요.에이비는.생각 수도 천을 될 거야.그걸 ? 절
epoch 9
Epoch 1/1
122483/122483 [==============================

렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨하고 비씨를 구하라는 얘기예요. 그러면 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이니까 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이다. 이렇게 여러분이 구할 수 있겠죠. 그렇죠? 그러면 이 문제 삼각형 넓이 구하는 거잖아요, 그렇죠? 그러면 이 각은 엑스라는 건 에이비라는 건 이 길이가 육이라고 했어요. 그러면 이 각은 엑스라는 건 에이비를 구하라는 얘기예요. 그러면 이 길이가 삼각형이 있고 이 각이 엑스면 이 각이 사십오도 이렇게 되겠네요. 그렇죠? 그러면 이 길이가 삼이고 이 각은 이 각이 엑스면 이 각이 엑스면 이 각이 엑스면 이 각이 엑스면 이 각의 십오도이면 이 각이 엑스면 이 
------ 온도: 0.5
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨하고 씨에이치를 구해서 비디를 구하라는 얘기예요. 그러면 이 길이가 삼십도를 이용하여 이라는 얘기야. 그다음에 이 변의 길이가 삼이고 이 길이가 삼이고 이 길이가 삼이니까 이 길이가 일이면 이 각이 엑스면 이 각이 엑스면 이 각이 삼십도를 뭐로 가 볼 텐데 이 길이가 사인 삼십도죠? 결국 이분의일, 이분의이 모두 더한 게 육십도이서 이렇게 되겠죠. 그러면 이 길이가 삼이니까 이 길이가 루트삼에이치가 이렇게 되겠죠. 이 길이가 루트삼에이치가 십이. 이렇게 여러분이 답을 구할 수 있겠죠. 그렇죠? 그러니까 여러분이 정리해 보겠습니다. 이 삼각형에서 사각형 넓이를 구하라. 이 문제 이렇게 이 문제를 나오는 거예요. 그렇죠? 그러면 우리는 어야기 하면 결국 이 길이가 사인 사십오도가 삼분의일이죠. 이 문제 이렇게 
------ 온도: 1.0
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부

c:\users\user\miniconda3\envs\jjtensor\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


도겠겠죠.여기에서 배의 길이가 직각이면 일이 일이니까 되고 탄젠트 삼십도는 이분의루트삼이니까 오씨록은이에서 당연히 끝작아요, 잘 끝나. 누가 ? 분의 높이잖아. 그렇죠? 들어금 이 걸림할 수 있었다는 거야.그래서 엑스곱하기탄젠트 비를 에이치라고 해본 다 이렇게 여러분들해서 삼각비 구하겠구나.얼마일 딱 들한 거야? 삼루트이로 답을 단가 방법으로 구할 수 있습니다.그대로 이 길이를 구할 수 있다는 얘기야. 여기에 삼십루트이야. 그다음 림사의 만큼 하지만단국은 비, 삼삼각형의 디에
------ 온도: 1.2
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨라는 엑스라고 생각할 수 있으면 삼각형에이에이비에? 건였을 거 우리가 죠? 겠죠. 기준각이 안 알져 있다면 이제는 구하라는 얘기고, 그 따라서 엑스도의 비는 삼이에이분. 그러면 백이십도잖아. 했을 때 씨는 고코비? 잘 보세요, 여러분늘  답이 수 있다고 해 얼비? 육 탄젠트 정도는 게 해 한다고 의해 다? 뭐가 된다? 씨에이치까지점 삼이 써, 에? 갖고 테고. 드리면 엑스는 어떻게 할 수 있다?디치를 구하면 육은 무을 구곱하기 약막 돼, 되면 밑변의 길이가 나누구대한이네. 은 워. 그런 거 에이, 일도죠. 그렇죠?그예요. 에이씨디의 길이는 사루트다.뭐이 같아, 그것이었습니다.하입니다, 게 아이디어. 영점도이팔인두예요.여기서 한번 디씨가 육곱하기사인 팔십도곱하기만 높이 앞에서 엑스 엑스가 뭐예요? 씨에이치의 
epoch 18
Epoch 1/1
122483/122483 [==============================] - 98s 798us/step - loss: 1.5430
--- 시드 텍스트: "렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨"
------ 온도: 0.2
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨 구하고 싶어

렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨의 길이가 이루트이. 이렇게 되겠예요. 이 길이가 이루트삼이다. 이렇게 여러분들이 백팔십도를 이용하는 거예요. 그럼 이 길이가 삼이고 이 길이가 이루트삼이 나누기 이렇게 여러분들이 이 각을 이용하면 이 각이 구십도라고 해야겠죠. 그럼 우리는 이 길이 구하라는 얘기예요. 그러면 이 각은 우리가 그런데 이 길이가 같아요. 씨에이치에서 에이치라고 해서 에이에이치는 이 길이가 삼에이치에서 에이치를 구할 수 있다는 얘기고, 이 길이가 사인 에이가 뭐야? 이 삼각형에서 높이를 구할 수 있다는 얘기고, 삼각형에이비에이치에서 에이치라고 놔 보고 삼각형 에이비씨에서 비에이치하고 에이. 이렇게 나와 있고 에이에이치를 구하래. 그러면 이 길이가 이대있높이. 그럼 우리는 이 각을 엑스면 이 각에 대해서 에이씨는 코사인 에이가 이
------ 온도: 1.0
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부터 가보면 ‘나 에이씨는이곱하기사인 공식이 아니에요, 삼분의루트이구나. 그런데 각의 곱하기이 거리에 빼 주면 되는 거예요. 그렇죠?에이씨의 길이는 영점삼이가 되니까 엑스분의이루트삼을 더하면 몇 에들은 도로 삼십칠로. 코사인 사십도구나. 이렇게 여러분들이 세 번째 기 단위는 일반 십이의 제곱이었어요. 왔는데 이. 그런데 외우는 거 선생님이 들 들어드렸만나면.반지름의 길이가 ’이면 오는 넓이를 구하고 싶은 거예요. 빗변이에요. 함께 풀어이죠.‘선생님, 여기에서 타고 보이, 이 같이 문 보세요. 이 삼십도 두 변의 길이였건데 대이 할 수 있다는 이 기가, 그죠, 여기 육십도 이십부다른 직삼각형이 나머지 사각형이죠? 에이의 값이 사인 탄젠트 기준 각에 답드억이얼마입니다, 그래요, 에이의 값. 다 코사인 육십도. 이분의루트이만 해주면 되
------ 온도: 1.2
렇게 수선을 그어줄게요. 그래서 수선의 발을 에이치라고 한번 놔보겠습니다.그러면 결론부

## Save model

In [105]:
# !mkdir -p saved_model
# model.save('saved_model/EBSepoch30lstm') 

In [7]:
import tensorflow as tf
new_model = tf.keras.models.load_model('saved_model/EBSepoch30lstm')

# 모델 구조를 확인합니다
new_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               461312    
_________________________________________________________________
dense_3 (Dense)              (None, 772)               99588     
Total params: 560,900
Trainable params: 560,900
Non-trainable params: 0
_________________________________________________________________


In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(ko_pre_EBS) - maxlen, step):
    sentences.append(ko_pre_EBS[i: i + maxlen])
    next_chars.append(ko_pre_EBS[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(ko_pre_EBS)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 122483
고유한 글자: 772
벡터화...


In [22]:
test = '성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 했으니까 닮은 직각삼각형 그려 봐야되 겠죠 닮은 직각삼각형 개 편하게 그릴수있는 제일 좋은 방법 뭘 까 생각 해보면우리 는 그리면 될거 같아요'
test[0:60]

'성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형'

In [24]:
maxlen = 60
# 무작위로 시드 텍스트를 선택합니다
seed_text = test[0:60]

# 여러가지 샘플링 온도를 시도합니다
for temperature in np.linspace(0.1,1,10):
    print('------ 온도:', temperature)
    generated_text = seed_text
    sys.stdout.write(generated_text)

    # 시드 텍스트에서 시작해서 40개의 글자를 생성합니다
    for i in range(40):
        # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        # 다음 글자를 샘플링합니다
        preds = new_model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

# test = '성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형
# 했으니까 닮은 직각삼각형 그려 봐야되 겠죠 닮은 직각삼각형 개 편하게 그릴수있는 제일 좋은 방법 뭘 까 생각 해보면
# 우리 는 그리면 될거 같아요'


------ 온도: 0.1
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 에이비씨에서 삼십도를 구하면 되는 거예요. 그러면 이 길이가 이루트이를
------ 온도: 0.2
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 에이비씨에서 삼십도를 구하면 되는 거예요. 그렇죠? 그러면 이 길이도 
------ 온도: 0.30000000000000004
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 에이비씨에서 삼십도를 이야기해요. 그렇죠? 그러니까 이 길이도 에이치 
------ 온도: 0.4
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 그리면 안 돼. 이 각 뭐예요? 빗변이 뭐예요? 밑변이 주어져 있어요.
------ 온도: 0.5
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 그리면 두 변이 아닌지 하면 이 길이를 구하고자 하는 거예요. 그럼 우
------ 온도: 0.6
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형에서의 한 변 구한다. 이제 다는 영도 이렇게 그려주어 보는 문제예요. 
------ 온도: 0.7000000000000001
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형이 아니야. 그렇죠? 이 길이 중에 분의 높이, 일이 되는구나. 그러니까
------ 온도: 0.8
성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다 그렇다면 닮은 직각삼각형 에이비씨에서 맞은 ‘우빗변 분의 높이의 값도 사각형이니까, 마지막 문제
------ 온도: 0.9
성질 알고가자는 삼각비 공부 하기에 앞선 징 검